In [ ]:
import numpy as np
import pandas as pd
import configparser 
from sklearn import utils

In [ ]:
from ipynb.fs.full.ImportExportData import exportDataframeToCSV,importCsvToDataframe
from ipynb.fs.full.EdfManipulation import read_and_store_data
from ipynb.fs.full.ClassificationPerformanceIndexes import classificationPerformanceIndexes
from ipynb.fs.full.FeatureClassificationMethods import SVM_Method, SVM_Kfold_Method, KNN_Method, KNN_Kfold_Method
from ipynb.fs.full.FeatureComputation import leftRightHemisphericChannels, featureExtraction
from ipynb.fs.full.FeatureNormalization import featureNormalization
from ipynb.fs.full.FeatureSelection import dimentionalityReduction
from ipynb.fs.full.DatasetBalancing import oversamplingSMOTE
from ipynb.fs.full.SplitDataset import createTrainingAndTestDatasets

## Main program

### Parameters

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')

In [ ]:
dataset = config['DATASET'].get('dataset')
sample_rate = config['DATASET'].getint('sample_rate')
time_window = config['DATASET'].getint('time_window')
test_ratio = config['TRAINING'].getfloat('test_ratio')
pca_tolerance = config['PCA'].getfloat('pca_tolerance')
smote_neighbors = config['SMOTE'].getint('smote_neighbors')
k_fold = config['CLASSIFICATION'].getint('k_fold')
knn_neighbors = config['CLASSIFICATION'].getint('knn_neighbors')
step = time_window*sample_rate

### Seazure Detection Program

In [ ]:
df = read_and_store_data(dataset, sample_rate)
df = leftRightHemisphericChannels(df)
features = pd.DataFrame(featureExtraction (df, sample_rate, step))
del df
features = featureNormalization(features)
reduced_features = dimentionalityReduction(features, pca_tolerance)
del features
smote_features = utils.shuffle(oversamplingSMOTE(reduced_features.drop('seizure', axis = 1), reduced_features['seizure'], smote_neighbors))
smote_features.reset_index(drop = True, inplace = True)
del reduced_features

train_dat, test_dat, train_ind, test_ind = createTrainingAndTestDatasets(smote_features, test_ratio)
perfInd = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1 Score', 'MCC', 'Kappa']
results = pd.DataFrame(columns=perfInd)

#SVM
SVM_Method(train_dat, train_ind, test_dat, test_ind, results)
SVM_Kfold_Method(smote_features, k_fold, perfInd, results)

#KNN
KNN_Method(train_dat, train_ind, test_dat, test_ind, knn_neighbors, results)
KNN_Kfold_Method(smote_features, knn_neighbors, k_fold, perfInd, results)